- 참고 : https://sieon-dev.tistory.com/15

## 1. 모듈 import (필요없는 모듈 삭제, 필요한 모듈 추가하면서 진행)
- pandas
- numpy
- matplotlib
- re
- konlpy
- tensorflow
- seaborn
- pandas_profiling

In [1]:
pip install konlpy

     |████████████████████████████████| 19.4 MB 465 kB/s 
     |████████████████████████████████| 448 kB 58.9 MB/s 


In [2]:
# Colab에 Mecab 설치
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab190912.sh

Cloning into 'Mecab-ko-for-Google-Colab'...
remote: Enumerating objects: 109, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 109 (delta 7), reused 10 (delta 3), pack-reused 91
Receiving objects: 100% (109/109), 1.27 MiB | 20.97 MiB/s, done.
Resolving deltas: 100% (46/46), done.
/content/Mecab-ko-for-Google-Colab
Installing konlpy.....
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2022-03-28 05:45:51--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 104.192.141.1, 2406:da00:ff00::3403:4be7, 2406:da00:ff00::6b17:d1f5, ...
Connecting to bitbucket.org (bitbucket.org)|104.192.141.1|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://bbuseruploads.s3.amazonaws.com/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz?Signature=6RVDJ

In [3]:
pip install exchange-calendars

     |████████████████████████████████| 150 kB 15.0 MB/s 
  Created wheel for exchange-calendars: filename=exchange_calendars-3.6.1-py3-none-any.whl size=180505 sha256=f61a57ec0fdf807008f0cd6e41991c582ab569fe58b533bc9ef338e2e64d90af
  Stored in directory: /root/.cache/pip/wheels/db/70/a2/5e1d8d0873feb8cb9808f3b55e8f270698b742db54a4b8ad2e
Successfully built exchange-calendars


In [4]:
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from konlpy.tag import Okt
from konlpy.tag import Mecab
import MeCab
from tqdm import tqdm
import tensorflow
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import warnings
warnings.filterwarnings("ignore")
import seaborn as sns
import pandas_profiling
import sklearn
from sklearn.model_selection import train_test_split
import collections
from collections import Counter
import exchange_calendars as ecals
import datetime
import time
from wordcloud import WordCloud
import matplotlib as mpl
import nltk

### 1-1. 파이썬 & 모듈 버전 확인 (필요없는 모듈 삭제, 필요한 모듈 추가하면서 진행)

In [5]:
print(" python version : ", sys.version, "\n", "-" * 100)
print(" pandas version : ", pd.__version__, "\n", "-" * 100)
print(" numpy version : ", np.__version__, "\n", "-" * 100)
print(" re version : ", re.__version__, "\n", "-" * 100)
print(" tensorflow version : ", tensorflow.__version__, "\n", "-" * 100)
print(" seaborn version : ", sns.__version__, "\n", "-" * 100)
print(" sklearn version : ", sklearn.__version__, "\n", "-" * 100)
print(" exhange_calendars version : ", ecals.__version__, "\n", "-" * 100)
print(" nltk version : ", nltk.__version__, "\n", "-" * 100)

 python version :  3.7.13 (default, Mar 16 2022, 17:37:17) 
[GCC 7.5.0] 
 ----------------------------------------------------------------------------------------------------
 pandas version :  1.3.5 
 ----------------------------------------------------------------------------------------------------
 numpy version :  1.21.5 
 ----------------------------------------------------------------------------------------------------
 re version :  2.2.1 
 ----------------------------------------------------------------------------------------------------
 tensorflow version :  2.8.0 
 ----------------------------------------------------------------------------------------------------
 seaborn version :  0.11.2 
 ----------------------------------------------------------------------------------------------------
 sklearn version :  1.0.2 
 ----------------------------------------------------------------------------------------------------
 exhange_calendars version :  3.6.1 
 ----------------

### 1-2. 컴퓨터 사양 확인

![](2022-03-24-13-31-45.png)

## 2. 데이터 불러오기

In [6]:
from google.colab import files
up = files.upload()

Saving 005290.xlsx to 005290.xlsx


In [7]:
up = files.upload()

Saving Newsfile3.csv to Newsfile3.csv


In [8]:
# 뉴스데이터 불러오기
newsdata = pd.read_csv('Newsfile3.csv', encoding='euc-kr')
newsdata.tail(3)

,date,title
58439,20151019,"[특징주]제주반도체, 美·中 대규모 투자유치에 '上'"
58440,20151019,[반도체] 2016 반도체 투자 감소 예상 - 미래
58441,20151019,"제주반도체, '해외투자유치설' 조회공시 요구"


### 2-1. 휴장일 데이터 매칭

In [9]:
# newsdata의 datatype 확인
newsdata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58442 entries, 0 to 58441
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   date    58442 non-null  int64 
 1   title   58442 non-null  object
dtypes: int64(1), object(1)
memory usage: 913.3+ KB


- type 변환

In [10]:
# datatype : int -> string
newsdata = newsdata.astype({'date':'string'})
newsdata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58442 entries, 0 to 58441
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   date    58442 non-null  string
 1   title   58442 non-null  object
dtypes: object(1), string(1)
memory usage: 913.3+ KB


In [11]:
# datatype : string -> datetime
newsdata['date'] = pd.to_datetime(newsdata['date'])
newsdata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58442 entries, 0 to 58441
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    58442 non-null  datetime64[ns]
 1   title   58442 non-null  object        
dtypes: datetime64[ns](1), object(1)
memory usage: 913.3+ KB


- 뉴스데이터의 처음 날짜와 마지막 날짜 확인

In [12]:
first_date = newsdata['date'][0]
last_date = newsdata['date'][-1:]
print(first_date, last_date)

2022-03-16 00:00:00 58441   2015-10-19
Name: date, dtype: datetime64[ns]


In [13]:
# newsdata -> newsdata_all : 처음 날짜와 마지막 날짜가 포함된 데이터로 이름 변결
newsdata_all = newsdata[newsdata['date'].between('2015-10-19', '2022-03-16')]
newsdata_all

,date,title
0,2022-03-16,"유럽 반도체 공급망 지원 호응… 인텔, 10년간 110조 통큰 투자"
1,2022-03-16,반도체 또다시 ‘쩐의 전쟁‘…인텔 “유럽 전역에 110조 투자”
2,2022-03-16,"“韓, 세계 2위 中수입시장 비중 줄어…반도체 외 돌파구 필요”"
3,2022-03-16,"[특징주] 코리아써키트, 반도체 패키지기판 성장 기대감에 5%↑"
4,2022-03-16,"[특징주] 반도체 대장주 강세…SK하이닉스, 2% 넘게 상승"
...,...,...
58437,2015-10-19,세계 반도체 M&A 1006억달러
58438,2015-10-19,반도체社 인수·합병 봇물…올 들어 1000억달러 돌파
58439,2015-10-19,"[특징주]제주반도체, 美·中 대규모 투자유치에 '上'"
58440,2015-10-19,[반도체] 2016 반도체 투자 감소 예상 - 미래


- 리스트로 변환

In [14]:
# 'date' 컬럼 리스트로 변경
news = newsdata_all['date'].to_list()
news[:5]

[Timestamp('2022-03-16 00:00:00'),
 Timestamp('2022-03-16 00:00:00'),
 Timestamp('2022-03-16 00:00:00'),
 Timestamp('2022-03-16 00:00:00'),
 Timestamp('2022-03-16 00:00:00')]

### 2-2. 한국 거래소 개장일 확인

In [15]:
# 한국코드
XKRX = ecals.get_calendar("XKRX")       
# 2021-01-01은 개장일인지 확인 : 'False' -> 휴장일 , 'True' -> 개장일
print("2021-01-01 개장 확인: ", XKRX.is_session('2021-01-01'))    
# 오늘(22-03-24)은 개장일인지 확인 :'False' -> 휴장일 , 'True' -> 개장일
print("오늘날짜 개장일 확인: ", XKRX.is_session(datetime.date.today().strftime('%Y-%m-%d')))  
# 다음 개장일은 언제인지 확인
print("다음 개장일 날짜 확인: ", XKRX.next_open(pd.Timestamp.today().strftime('%Y-%m-%d')))

2021-01-01 개장 확인:  False
오늘날짜 개장일 확인:  True
다음 개장일 날짜 확인:  2022-03-29 00:00:00+00:00


In [16]:
# 뉴스데이터 날짜 다음날에 해당하는 개장일 날짜 확인
next_open_list = []

for day in news:
    next_open_date = XKRX.next_open(day)
    next_open_list.append(next_open_date)

In [17]:
next_open_list[:5]

[Timestamp('2022-03-17 00:00:00+0000', tz='UTC'),
 Timestamp('2022-03-17 00:00:00+0000', tz='UTC'),
 Timestamp('2022-03-17 00:00:00+0000', tz='UTC'),
 Timestamp('2022-03-17 00:00:00+0000', tz='UTC'),
 Timestamp('2022-03-17 00:00:00+0000', tz='UTC')]

In [18]:
# 뉴스나온 날짜 다음 개장일 컬럼 추가
newsdata_all['NextOpenDate'] = next_open_list
newsdata_all.head(3)

,date,title,NextOpenDate
0,2022-03-16,"유럽 반도체 공급망 지원 호응… 인텔, 10년간 110조 통큰 투자",2022-03-17 00:00:00+00:00
1,2022-03-16,반도체 또다시 ‘쩐의 전쟁‘…인텔 “유럽 전역에 110조 투자”,2022-03-17 00:00:00+00:00
2,2022-03-16,"“韓, 세계 2위 中수입시장 비중 줄어…반도체 외 돌파구 필요”",2022-03-17 00:00:00+00:00


In [19]:
newsdata_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 58442 entries, 0 to 58441
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype              
---  ------        --------------  -----              
 0   date          58442 non-null  datetime64[ns]     
 1   title         58442 non-null  object             
 2   NextOpenDate  58442 non-null  datetime64[ns, UTC]
dtypes: datetime64[ns, UTC](1), datetime64[ns](1), object(1)
memory usage: 1.8+ MB


- type 변환

In [20]:
newsdata_all = newsdata_all.astype({'NextOpenDate':'string'})
newsdata_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 58442 entries, 0 to 58441
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   date          58442 non-null  datetime64[ns]
 1   title         58442 non-null  object        
 2   NextOpenDate  58442 non-null  string        
dtypes: datetime64[ns](1), object(1), string(1)
memory usage: 1.8+ MB


In [21]:
a = newsdata_all['NextOpenDate'][0]
b = a.split(' ')[0]
b

'2022-03-17'

In [22]:
newsdata_all['NextOpenDate'] = newsdata_all['NextOpenDate'].apply(lambda x: x.split(' ')[0])
newsdata_all.head(3)

,date,title,NextOpenDate
0,2022-03-16,"유럽 반도체 공급망 지원 호응… 인텔, 10년간 110조 통큰 투자",2022-03-17
1,2022-03-16,반도체 또다시 ‘쩐의 전쟁‘…인텔 “유럽 전역에 110조 투자”,2022-03-17
2,2022-03-16,"“韓, 세계 2위 中수입시장 비중 줄어…반도체 외 돌파구 필요”",2022-03-17


In [23]:
newsdata_all['NextOpenDate'] = pd.to_datetime(newsdata_all['NextOpenDate'])
newsdata_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 58442 entries, 0 to 58441
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   date          58442 non-null  datetime64[ns]
 1   title         58442 non-null  object        
 2   NextOpenDate  58442 non-null  datetime64[ns]
dtypes: datetime64[ns](2), object(1)
memory usage: 3.8+ MB


In [24]:
# 개장일 기준 'Year' 컬럼 추가
newsdata_all['NextOpenDateYear'] = newsdata_all['NextOpenDate'].dt.year
newsdata_all.head(3)

,date,title,NextOpenDate,NextOpenDateYear
0,2022-03-16,"유럽 반도체 공급망 지원 호응… 인텔, 10년간 110조 통큰 투자",2022-03-17,2022
1,2022-03-16,반도체 또다시 ‘쩐의 전쟁‘…인텔 “유럽 전역에 110조 투자”,2022-03-17,2022
2,2022-03-16,"“韓, 세계 2위 中수입시장 비중 줄어…반도체 외 돌파구 필요”",2022-03-17,2022


In [25]:
# col의 인덱스 변경
col = newsdata_all.columns.to_numpy()
col

array(['date', 'title', 'NextOpenDate', 'NextOpenDateYear'], dtype=object)

In [26]:
col = col[[0,3,2,1]]
col

array(['date', 'NextOpenDateYear', 'NextOpenDate', 'title'], dtype=object)

In [27]:
newsdata_all = newsdata_all[col]
newsdata_all.head(3)

,date,NextOpenDateYear,NextOpenDate,title
0,2022-03-16,2022,2022-03-17,"유럽 반도체 공급망 지원 호응… 인텔, 10년간 110조 통큰 투자"
1,2022-03-16,2022,2022-03-17,반도체 또다시 ‘쩐의 전쟁‘…인텔 “유럽 전역에 110조 투자”
2,2022-03-16,2022,2022-03-17,"“韓, 세계 2위 中수입시장 비중 줄어…반도체 외 돌파구 필요”"


## 3. 데이터 확인

In [28]:
# 데이터 갯수 : 58442개
# 컬럼 갯수 : 4개
newsdata_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 58442 entries, 0 to 58441
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   date              58442 non-null  datetime64[ns]
 1   NextOpenDateYear  58442 non-null  int64         
 2   NextOpenDate      58442 non-null  datetime64[ns]
 3   title             58442 non-null  object        
dtypes: datetime64[ns](2), int64(1), object(1)
memory usage: 4.2+ MB


- 결측치 확인

In [29]:
# 결측치 확인
newsdata_all.isnull().sum()

date                0
NextOpenDateYear    0
NextOpenDate        0
title               0
dtype: int64

## 4. 기업별 '현재가', '당일종가-어제종가', '상승률', 'label' 데이터와 뉴스데이터 병합

### 4-1. 반도체 기업 데이터 불러오기 : 005290

In [30]:
df1 = pd.read_excel('005290.xlsx')
df1.head(2)

,Unnamed: 0,일자,현재가,당일종가-어제종가,상승률
0,0,20220314,31900,-1500,-0.047022
1,1,20220311,33400,-300,-0.008982


In [31]:
df1.drop(columns=['Unnamed: 0'], inplace=True)
df1.head(2)

,일자,현재가,당일종가-어제종가,상승률
0,20220314,31900,-1500,-0.047022
1,20220311,33400,-300,-0.008982


- 긍정 & 부정 라벨링
    - 상승률이 0보다 크면 `1`로 매칭
    - 상승률이 0보다 작으면 `0`으로 매칭

In [32]:
df1['label'] = df1.apply(lambda x: 1 if x['상승률'] >= 0 else 0, axis=1)
df1

,일자,현재가,당일종가-어제종가,상승률,label
0,20220314,31900,-1500,-0.047022,0
1,20220311,33400,-300,-0.008982,0
2,20220310,33700,550,0.016320,1
3,20220308,33150,-800,-0.024133,0
4,20220307,33950,-1700,-0.050074,0
...,...,...,...,...,...
5479,19991227,8800,940,0.106818,1
5480,19991224,7860,840,0.106870,1
5481,19991223,7020,750,0.106838,1
5482,19991222,6270,670,0.106858,1


- 라벨링 분포 확인

In [33]:
sns.countplot(x='label', data=df1)

- data type 변경

In [34]:
# data type : int -> string
df1 = df1.astype({'일자':'string'})
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5484 entries, 0 to 5483
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   일자         5484 non-null   string 
 1   현재가        5484 non-null   int64  
 2   당일종가-어제종가  5484 non-null   int64  
 3   상승률        5484 non-null   float64
 4   label      5484 non-null   int64  
dtypes: float64(1), int64(3), string(1)
memory usage: 214.3 KB


In [35]:
# data type : string -> datetime
df1['일자'] = pd.to_datetime(df1['일자'])
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5484 entries, 0 to 5483
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   일자         5484 non-null   datetime64[ns]
 1   현재가        5484 non-null   int64         
 2   당일종가-어제종가  5484 non-null   int64         
 3   상승률        5484 non-null   float64       
 4   label      5484 non-null   int64         
dtypes: datetime64[ns](1), float64(1), int64(3)
memory usage: 214.3 KB


### 4-2. 뉴스데이터 컬럼명 변경

In [36]:
# 덕산네오룩스의 데이터프레임과 병합하기 위해 컬럼명 변경
newsdata_all.rename(columns={'NextOpenDate':'일자'}, inplace=True)
newsdata_all.head(2)

,date,NextOpenDateYear,일자,title
0,2022-03-16,2022,2022-03-17,"유럽 반도체 공급망 지원 호응… 인텔, 10년간 110조 통큰 투자"
1,2022-03-16,2022,2022-03-17,반도체 또다시 ‘쩐의 전쟁‘…인텔 “유럽 전역에 110조 투자”


### 4-3. 뉴스데이터와 덕산네오룩스 주가데이터 병합

In [37]:
merge_data = pd.merge(newsdata_all, df1, on='일자', how='inner')
merge_data.head(2)

,date,NextOpenDateYear,일자,title,현재가,당일종가-어제종가,상승률,label
0,2022-03-13,2022,2022-03-14,전함 포템킨·오데사·네온사인…반도체는 왜 또 위기일까[인더독],31900,-1500,-0.047022,0
1,2022-03-13,2022,2022-03-14,"한국기업, 반도체·전자 부품 조달 ‘빨간불’",31900,-1500,-0.047022,0


- '일자', 'title', 'label'만 남긴 나머지 컬럼 삭제

In [38]:
merge_data = merge_data[['일자', 'title', 'label']]
merge_data.head(5)

,일자,title,label
0,2022-03-14,전함 포템킨·오데사·네온사인…반도체는 왜 또 위기일까[인더독],0
1,2022-03-14,"한국기업, 반도체·전자 부품 조달 ‘빨간불’",0
2,2022-03-14,“삼성·SK만 잘하면 뭐해?” 한국 반도체 진짜 ‘약점’이 바로 [비즈360],0
3,2022-03-14,"""반도체 일본꼴 날 수도…수도권 대학 반도체학과 정원 풀어야"" [당선인에 바란다]",0
4,2022-03-14,"[우크라 침공] 러시아, 반도체소자 등 500개 품목 대외수출 금지·제한",0


## 5. 데이터 전처리
    - 한글 이외의 문자열은 빈칸(`''`)처리
    - 품사 중 명사만 추출하여 'nouns' 컬럼에 삽입

In [39]:
okt = Okt()
n_ = []
title_rename = []

for i in range(len(merge_data)):
    if (i % 10000 == 0):
        print(i, "단계 완료")
    title_rename.append(re.sub("[\(\[].*?[\)\]]", "", merge_data.iloc[i]['title']))
    n_.append(' '.join(okt.nouns(merge_data.iloc[i]['title'])))
merge_data['nouns'] = n_
merge_data['title'] = title_rename
merge_data = merge_data[merge_data['nouns'] != '']

# 2차 불용어 제거
merge_data['title'] = merge_data['title'].str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣]','')
merge_data['title'].replace('', np.nan, inplace=True)
merge_data = merge_data.dropna(how='any')

0 단계 완료
10000 단계 완료
20000 단계 완료
30000 단계 완료
40000 단계 완료
50000 단계 완료


In [40]:
merge_data.head(3)

,일자,title,label,nouns
0,2022-03-14,전함포템킨오데사네온사인반도체는왜또위기일까,0,전함 포템킨 오데사 네온사인 반도체 왜 또 위기 인 더독
1,2022-03-14,한국기업반도체전자부품조달빨간불,0,한국 기업 반도체 전자 부품 조달 불
2,2022-03-14,삼성만잘하면뭐해한국반도체진짜약점이바로,0,삼성 만 뭐 한국 반도체 진짜 약점 이 바로 비즈


In [41]:
news_labeling = merge_data.copy()

In [42]:
# 결측치 확인
news_labeling.dropna(how='any', inplace=True)
news_labeling.shape

(58343, 4)

- label 분포 확인

In [44]:
# 라벨링 값 개수 확인
news_labeling['label'].value_counts()

1    29202
0    29141
Name: label, dtype: int64

## 6. 감성사전 구축

- 한 글자 단어 제거, 두 글자 이상인 단어들의 점수 초기화

## 6-1. train_data, test_data 분할

In [54]:
# train, test set 분리
df_train, df_test = train_test_split(news_labeling)

In [50]:
vocab = {}
cnt = 0
for i in news_labeling['nouns']:
    i = i.split(' ')
    for j in range(len(i)):
        if i[j] in vocab or len(i[j]) <= 1:
            cnt = cnt+1
            pass
        else:
            vocab[i[j]] = 0
print(len(vocab), vocab)

9999 {'전함': 0, '포템킨': 0, '오데사': 0, '네온사인': 0, '반도체': 0, '위기': 0, '더독': 0, '한국': 0, '기업': 0, '전자': 0, '부품': 0, '조달': 0, '삼성': 0, '진짜': 0, '약점': 0, '바로': 0, '비즈': 0, '일본': 0, '수도': 0, '수도권': 0, '대학': 0, '학과': 0, '정원': 0, '당선인': 0, '침공': 0, '러시아': 0, '반도체소자': 0, '품목': 0, '대외': 0, '수출': 0, '금지': 0, '제한': 0, '호국': 0, '통제': 0, '공급': 0, '우려': 0, '애플': 0, '수혜': 0, '가능성': 0, '전기차': 0, '원가': 0, '상승': 0, '비상': 0, '배터리': 0, '철강': 0, '고공': 0, '행진': 0, '경제': 0, '제재': 0, '보복': 0, '시작': 0, '소자': 0, '속보': 0, '발표': 0, '포함': 0, '전쟁': 0, '가스': 0, '생산': 0, '중단': 0, '네온': 0, '절반': 0, '대란': 0, '심화': 0, '파문': 0, '대전': 0, '조짐': 0, '격화': 0, '인텔': 0, '계속': 0, '윤석열': 0, '공약': 0, '실현': 0, '집적회로': 0, '상보': 0, '산다': 0, '두산': 0, '통큰': 0, '강행': 0, '이유': 0, '브로드컴': 0, '주주': 0, '환원': 0, '기대': 0, '조치': 0, '여파': 0, '변신': 0, '승부': 0, '과연': 0, '자등': 0, '인력': 0, '양성': 0, '기금': 0, '조성': 0, '도로': 0, '지수': 0, '하락': 0, '악재': 0, '코스피': 0, '약세': 0, '마감': 0, '백악관': 0, '초대': 0, '바이든': 0, '텍사스': 0, '투자': 0, '감사': 0, '세계': 0, '위도': 0,

In [62]:
up = 29202
down = 29141
up_ratio = up/(up+down)
down_ratio = down/(up+down)

for i, w in enumerate(news_labeling['nouns']):
    w = w.split(' ')
    if (news_labeling.iloc[i]['label'] == 1):
        for j in range(len(w)):
            noun = w[j]
            if len(noun) <= 1:
                continue
            vocab[noun] = vocab[noun] + up_ratio
        else:
            for j in range(len(w)):
                noun = w[j]
                if len(noun) <= 1:
                    continue
                vocab[noun] = vocab[noun] - down_ratio

In [63]:
vocab

{'전함': 0,
 '포템킨': 0,
 '오데사': 0,
 '네온사인': 0,
 '반도체': 63.555010883731406,
 '위기': 0.6837838301081308,
 '더독': 0.00418216409852068,
 '한국': 1.511852321615117,
 '기업': 2.0910820492601068,
 '전자': 9.560427129221182,
 '부품': 0.7506984556844617,
 '조달': 0.1150095127093187,
 '삼성': 13.698678504710037,
 '진짜': 0.03973055893594646,
 '약점': 0.00627324614778102,
 '바로': 0.01254649229556204,
 '비즈': 0.08991652811819462,
 '일본': 0.6189602865810605,
 '수도': 0.1359203332019221,
 '수도권': 0.03763947688668612,
 '대학': 0.0627324614778102,
 '학과': 0.07946111787189292,
 '정원': 0.0104554102463017,
 '당선인': 0,
 '침공': 0.02509298459112408,
 '러시아': 0.06482354352707054,
 '반도체소자': 0.00209108204926034,
 '품목': 0.26974958435458385,
 '대외': 0.0522770512315085,
 '수출': 6.386164578441989,
 '금지': 0.0627324614778102,
 '제한': 0.2049260408275133,
 '호국': 0,
 '통제': 0.0418216409852068,
 '공급': 4.240714395899486,
 '우려': 0.7674271120785446,
 '애플': 0.3345731278816544,
 '수혜': 0.4809488713298782,
 '가능성': 0.23420118951715807,
 '전기차': 0.3366642099309147,
 

### 6-2. train_data

In [69]:
# 감성사전을 활용하여 뉴스의 평균 감성 점수를 산출한 뒤 컬럼에 추가 
total = []
for i,w in enumerate(df_train['nouns']):
    sent_score = 0
    w = w.split(' ')
    for j in w:
        if (len(j) <= 1):
            continue
        elif (j not in vocab):
            continue
        else:
            sent_score = sent_score + vocab[j]
    total.append(sent_score/len(w))
df_train['sent_score'] = total

In [72]:
df_train.head()

,일자,title,label,nouns,sent_score
42989,2018-05-24,반도체업종에또미국훈풍,0,반도체 업종 또 미국 훈풍,13.030938
16848,2021-02-08,반도체품귀에생산차질완성차업계긴장,0,반도체 품귀 생산 차질 완성 차 업계 긴장,8.490149
38780,2018-11-21,아이텍반도체상승,1,아이 텍 반도체 상승,16.326834
29913,2019-08-05,대성장률적신호수출규제에반도체직격탄,1,성장 률 적신호 수출 규제 반도체 직격탄,10.328559
49751,2017-07-05,삼성반도체디스플레이에조원투자넘사벽굳히기,1,삼성 반도체 디스플레이 투자 넘사벽,16.638891


In [73]:
# sent_score가 감성사전의 평균 점수보다 높으면 1, 낮으면 0으로 새롭게 라벨링
sum = 0
for i in range(len(vocab)):
    sum = sum + list(vocab.values())[i]
sent_mean = sum/len(vocab)

a_ = []
for i in range(len(df_train)):
    if (df_train.iloc[i]['sent_score'] > sent_mean):
        a_.append(1)
    else : 
        a_.append(0)
df_train['sent_label'] = a_

In [74]:
df_train.head()

,일자,title,label,nouns,sent_score,sent_label
42989,2018-05-24,반도체업종에또미국훈풍,0,반도체 업종 또 미국 훈풍,13.030938,1
16848,2021-02-08,반도체품귀에생산차질완성차업계긴장,0,반도체 품귀 생산 차질 완성 차 업계 긴장,8.490149,1
38780,2018-11-21,아이텍반도체상승,1,아이 텍 반도체 상승,16.326834,1
29913,2019-08-05,대성장률적신호수출규제에반도체직격탄,1,성장 률 적신호 수출 규제 반도체 직격탄,10.328559,1
49751,2017-07-05,삼성반도체디스플레이에조원투자넘사벽굳히기,1,삼성 반도체 디스플레이 투자 넘사벽,16.638891,1


### 6-3. test_data

In [75]:
# 감성사전을 활용하여 뉴스의 평균 감성 점수를 산출한 뒤 컬럼에 추가 
total = []
for i,w in enumerate(df_test['nouns']):
    sent_score = 0
    w = w.split(' ')
    for j in w:
        if (len(j) <= 1):
            continue
        elif (j not in vocab):
            continue
        else:
            sent_score = sent_score + vocab[j]
    total.append(sent_score/len(w))
df_test['sent_score'] = total

In [76]:
# sent_score가 감성사전의 평균 점수보다 높으면 1, 낮으면 0으로 새롭게 라벨링
sum = 0
for i in range(len(vocab)):
    sum = sum + list(vocab.values())[i]
sent_mean = sum/len(vocab)

a_ = []
for i in range(len(df_test)):
    if (df_test.iloc[i]['sent_score'] > sent_mean):
        a_.append(1)
    else : 
        a_.append(0)
df_test['sent_label'] = a_

In [77]:
df_test.head()

,일자,title,label,nouns,sent_score,sent_label
43762,2018-04-17,반도체보고서국가기밀인지판단할산업부지만눈치보기급급,0,반도체 보고서 국가 기밀 판단 산업부 눈치 보기 급,7.159252,1
45800,2018-01-11,삼성전자작년영업이익조원역대최고반도체만조,1,삼성 전자 작년 영업 이익 역대 최고 반도체,11.583643,1
13960,2021-04-14,웨이퍼손에든바이든반도체전쟁선포반도체는인프라공격적투자필,0,웨이퍼 손 바이든 반도체 전쟁 선포 반도체 인프라 공격 투자 필,12.079748,1
17522,2021-01-25,전세계반도체부족국내충격도시간문제,1,전세계 반도체 부족 국내 충격 간문제,11.023962,1
10413,2021-06-11,반도체품귀가만든진풍경신차보다비싼중고차속출,1,반도체 품귀 진 풍경 신차 중고차 속출,9.145604,1


## 9. 모델링

- df_train

In [80]:
# validation set과 train set을 나눠준 뒤 형태소를 분류 -> X와 Y 값을 지정
train_data, validation_data = train_test_split(df_train, test_size=0.2, random_state=42)
train_data.dropna(how='any', inplace=True)
validation_data.dropna(how='any', inplace=True)
mecab = Mecab()
train_data['tokenized'] = train_data['title'].apply(mecab.morphs)
validation_data['tokenized'] = validation_data['title'].apply(mecab.morphs)
X_train = train_data['tokenized'].values
Y_train = train_data['sent_label'].values
X_vali = validation_data['tokenized'].values
Y_vali = validation_data['sent_label'].values

In [83]:
# keras의 Tokenizer를 사용해 텍스트를 시퀀스 벡터로 변환
vocab_size=3
tokenizer = Tokenizer(vocab_size, oov_token='OOV')
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_vali = tokenizer.texts_to_sequences(X_vali)

In [84]:
# 일정한 Input 형태를 만들기 위해 적절한 길이를 지정하고 padding으로 마무리
def below_threshold_len(max_len, nested_list):
    cnt = 0
    for s in nested_list:
        if (len(s) <= max_len):
            cnt = cnt + 1
    print('전체 샘플 중 길이가 %s 이하인 샘플의 비율 : %s'%(max_len, (cnt / len(nested_list))*100))

max_len = 20
below_threshold_len(max_len, X_train)
X_train = pad_sequences(X_train, maxlen = max_len)
X_vali = pad_sequences(X_vali, maxlen = max_len)

전체 샘플 중 길이가 20 이하인 샘플의 비율 : 99.99714326524783


In [100]:
# 모델 구현

from keras.models import Sequential
from keras.layers import Dense, Embedding, Bidirectional, LSTM
from keras.callbacks import EarlyStopping, ModelCheckpoint


model = Sequential()
model.add(Embedding(vocab_size, 100))
model.add(Bidirectional(LSTM(100)))
model.add(Dense(1, activation='sigmoid'))

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(X_train, Y_train, epochs=20, callbacks=[es, mc], batch_size=100, validation_split=0.2)

Epoch 1/20
280/281 [============================>.] - ETA: 0s - loss: 0.0085 - acc: 0.9970
Epoch 1: val_acc improved from -inf to 0.99986, saving model to best_model.h5
281/281 [==============================] - 25s 79ms/step - loss: 0.0085 - acc: 0.9970 - val_loss: 0.0020 - val_acc: 0.9999
Epoch 2/20
280/281 [============================>.] - ETA: 0s - loss: 0.0016 - acc: 0.9999
Epoch 2: val_acc did not improve from 0.99986
281/281 [==============================] - 21s 74ms/step - loss: 0.0016 - acc: 0.9999 - val_loss: 0.0016 - val_acc: 0.9999
Epoch 3/20
280/281 [============================>.] - ETA: 0s - loss: 0.0015 - acc: 0.9999
Epoch 3: val_acc did not improve from 0.99986
281/281 [==============================] - 21s 74ms/step - loss: 0.0015 - acc: 0.9999 - val_loss: 0.0018 - val_acc: 0.9999
Epoch 4/20
280/281 [============================>.] - ETA: 0s - loss: 0.0015 - acc: 0.9999
Epoch 4: val_acc did not improve from 0.99986
281/281 [==============================] - 20s 73ms

In [101]:
from keras.models import load_model

loaded_model = load_model('best_model.h5')

## 10. 예측

In [102]:
def sentiment_predict(new_sentence):
  new_sentence = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','', new_sentence)
  new_sentence = mecab.morphs(new_sentence) # 토큰화
  # new_sentence = [word for word in new_sentence if not word in stopwords] # 불용어 제거
  encoded = tokenizer.texts_to_sequences([new_sentence]) # 정수 인코딩
  pad_new = pad_sequences(encoded, maxlen = max_len) # 패딩
  score = float(loaded_model.predict(pad_new)) # 예측
  if(score > 0.5):
    print("{:.2f}% 확률로 긍정 리뷰입니다.".format(score * 100))
  else:
    print("{:.2f}% 확률로 부정 리뷰입니다.".format((1 - score) * 100))

In [103]:
sentiment_predict('메모리 반도체 바닥 다졌다...삼성전자·SK하이닉스 반등 기대감↑')

100.00% 확률로 긍정 리뷰입니다.


In [104]:
sentiment_predict('우크라 전쟁에 세계 반도체 대란 우려…"당장은 타격 제한적"')

100.00% 확률로 긍정 리뷰입니다.
